In [1]:
#importing libraries
from tensorflow import keras
 
from keras.preprocessing import text
from keras.utils import to_categorical  # Correct import

from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
# Define your generate_context_word_pairs function
def generate_context_word_pairs(corpus, window_size=2, vocab_size=None):
    context_length = window_size * 2

    for words in corpus:
        sentence_length = len(words)
       
        for index, word in enumerate(words):
            context_words = []
            label_word = []
            start = index - window_size
            end = index + window_size + 1

            context_words.append([words[i] for i in range(start, end) if 0 <= i < sentence_length and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, num_classes=vocab_size)  # Use to_categorical

            yield (x, y)

# Usage example
i = 0
for x, y in generate_context_word_pairs(wids, window_size=window_size, vocab_size=vocab_size):
    # Your code to process x and y goes here
    # You can remove the "if not in x[0]:" line as it's not necessary
    # Print statements for context and target are commented out
    # Adjust your processing logic here
    pass

    i += 1
    if i == 10:
        break


In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.7678847312927

Epoch: 2 	Loss: 429.6340391635895

Epoch: 3 	Loss: 426.5555248260498

Epoch: 4 	Loss: 423.462557554245

Epoch: 5 	Loss: 420.83976697921753



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.022259,-0.060878,0.016678,0.034775,-0.013786,0.043892,-0.046185,-0.039341,-0.036152,-0.051531,...,-0.021000,0.011192,-0.047044,-0.031826,-0.007379,-0.019068,-0.042662,-0.001140,-0.043065,-0.013345
networks,-0.045581,0.052467,0.057012,0.006601,0.058079,-0.053895,-0.001406,0.018578,-0.045625,-0.031360,...,0.021673,-0.017612,0.022579,-0.057040,-0.021848,-0.008740,-0.058819,-0.029761,0.052530,-0.032003
neural,-0.022761,-0.015829,0.048206,0.004180,0.026662,0.012026,0.024212,-0.009845,-0.003830,-0.040117,...,0.028783,-0.028019,0.034464,-0.024134,-0.041621,-0.041458,0.037261,-0.004421,-0.000345,-0.000150
and,0.000528,0.044162,-0.004411,0.008604,-0.035676,-0.014586,-0.005597,0.024556,-0.045509,0.030225,...,0.032072,0.039060,-0.048105,-0.035348,0.044256,0.029966,0.016837,-0.005986,-0.041124,0.004012
as,-0.014389,-0.008389,-0.008584,-0.005823,0.044693,0.005996,0.000427,0.006647,-0.024503,0.015301,...,-0.006592,0.049910,0.004097,0.041111,0.012217,0.006776,-0.034465,0.041075,0.006652,0.044156


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['inspection', 'recognition', 'medical', 'applied', 'reinforcement']}